#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)


In [2]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct

In [4]:
embeddings=(
    OllamaEmbeddings(model="llama3.2:1b")  ##by default it ues llama2
)

db=FAISS.from_documents(docs,embeddings)
db

In [5]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content


'We have borne with their present government through all these bitter months because of that friendshipâ€”exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance except from a lawless and malignant few.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [ ]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

Failed to get info from https://api.smith.langchain.com: LangSmithConnectionError('Connection error caused failure to GET /info in LangSmith API. Please confirm your internet connection. SSLError(MaxRetryError("HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Max retries exceeded with url: /info (Caused by SSLError(SSLCertVerificationError(1, \'[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1010)\')))"))\nContent-Length: None\nAPI Key: lsv2_********************************************e8')


'We have borne with their present government through all these bitter months because of that friendshipâ€”exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance except from a lawless and malignant few.'

Failed to batch ingest runs: langsmith.utils.LangSmithConnectionError: Connection error caused failure to POST https://api.smith.langchain.com/runs/batch in LangSmith API. Please confirm your internet connection. SSLError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs/batch (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1010)')))"))
Content-Length: 1123
API Key: lsv2_********************************************e8
post: trace=76515032-2757-4751-a70a-9665a8763356,id=76515032-2757-4751-a70a-9665a8763356
Failed to batch ingest runs: langsmith.utils.LangSmithConnectionError: Connection error caused failure to POST https://api.smith.langchain.com/runs/batch in LangSmith API. Please confirm your internet connection. SSLError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [7]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='16bb2a91-66c6-4d32-8b78-85d2935999b6', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendshipâ€”exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift 

In [8]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[-1.9710798263549805,
 2.7524361610412598,
 0.5358948111534119,
 -0.8852519392967224,
 4.516158580780029,
 -4.514031410217285,
 -0.9148536324501038,
 2.5720415115356445,
 -0.06580793112516403,
 1.4465779066085815,
 -0.7714748382568359,
 -2.3683879375457764,
 -0.6825301647186279,
 1.0715935230255127,
 2.9335591793060303,
 -0.9808536767959595,
 0.6848704814910889,
 0.9774548411369324,
 7.266448974609375,
 0.15200553834438324,
 -1.2033882141113281,
 1.7869967222213745,
 0.21478049457073212,
 -0.7093415260314941,
 -0.4089311957359314,
 3.2522354125976562,
 -3.644601345062256,
 2.628201961517334,
 -0.5708925724029541,
 1.9629452228546143,
 -0.47750407457351685,
 1.9503023624420166,
 -1.6354607343673706,
 2.036903142929077,
 0.24746550619602203,
 -0.3160436749458313,
 -1.1023333072662354,
 -2.1628074645996094,
 -0.49978259205818176,
 -1.0671405792236328,
 -1.5187023878097534,
 -0.26959365606307983,
 -0.45529109239578247,
 2.273359537124634,
 -0.2608479857444763,
 0.5995631217956543,
 1.93676

In [15]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='16bb2a91-66c6-4d32-8b78-85d2935999b6', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendshipâ€”exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift i

In [16]:
### Saving And Loading
db.save_local("faiss_index")

In [17]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [14]:
docs

[Document(id='16bb2a91-66c6-4d32-8b78-85d2935999b6', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendshipâ€”exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift i